In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.cuda.is_available()

True

In [2]:
from src.vlstm_v4.interface import qkvkernel

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/vlstm_v4/build.ninja...
Building extension module vlstm_v4...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt21cu118/bin/nvcc  -ccbin /home/max/miniconda3/envs/xlstmpt21cu118/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_v4 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include -isystem /home/max/miniconda3/envs/xlstmpt21cu118/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-c

Loading extension module vlstm_v4...


### qkvtest

In [24]:
S = 16 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 4 # dim per head
DTYPE = torch.float16
DEVICE = torch.device("cuda:0")

In [25]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 100.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
qs, qs.shape

(tensor([[[[0.0000, 0.0100, 0.0200, 0.0300],
           [0.0400, 0.0500, 0.0600, 0.0700],
           [0.0800, 0.0900, 0.1000, 0.1100],
           [0.1200, 0.1300, 0.1400, 0.1500],
           [0.1600, 0.1700, 0.1801, 0.1899],
           [0.2000, 0.2100, 0.2200, 0.2300],
           [0.2400, 0.2500, 0.2600, 0.2700],
           [0.2800, 0.2900, 0.3000, 0.3101],
           [0.3201, 0.3301, 0.3401, 0.3501],
           [0.3601, 0.3701, 0.3799, 0.3899],
           [0.3999, 0.4099, 0.4199, 0.4299],
           [0.4399, 0.4500, 0.4600, 0.4700],
           [0.4800, 0.4900, 0.5000, 0.5098],
           [0.5200, 0.5298, 0.5400, 0.5498],
           [0.5601, 0.5698, 0.5801, 0.5898],
           [0.6001, 0.6099, 0.6201, 0.6299]]]], device='cuda:0', dtype=torch.float16),
 torch.Size([1, 1, 16, 4]))

In [26]:
torch.set_printoptions(linewidth=200)
rs = qs @ ks.transpose(-1, -2)
rs, rs.shape

(tensor([[[[0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600, 0.0600],
           [0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200, 0.2200],
           [0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799, 0.3799],
           [0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400, 0.5400],
           [0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002, 0.7002],
           [0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599, 0.8599],
           [1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195, 1.0195],
           [1

In [27]:
# pytorch version
rs = qs @ ks.transpose(-1, -2) @ vs
rs, rs.shape

(tensor([[[[ 0.9600,  0.9600,  0.9600,  0.9600],
           [ 3.5195,  3.5195,  3.5195,  3.5195],
           [ 6.0781,  6.0781,  6.0781,  6.0781],
           [ 8.6406,  8.6406,  8.6406,  8.6406],
           [11.2031, 11.2031, 11.2031, 11.2031],
           [13.7578, 13.7578, 13.7578, 13.7578],
           [16.3125, 16.3125, 16.3125, 16.3125],
           [18.8750, 18.8750, 18.8750, 18.8750],
           [21.4375, 21.4375, 21.4375, 21.4375],
           [24.0000, 24.0000, 24.0000, 24.0000],
           [26.5625, 26.5625, 26.5625, 26.5625],
           [29.1250, 29.1250, 29.1250, 29.1250],
           [31.6719, 31.6719, 31.6719, 31.6719],
           [34.2500, 34.2500, 34.2500, 34.2500],
           [36.8125, 36.8125, 36.8125, 36.8125],
           [39.3750, 39.3750, 39.3750, 39.3750]]]], device='cuda:0', dtype=torch.float16),
 torch.Size([1, 1, 16, 4]))

In [28]:
# cuda kernel
rs = qkvkernel(mat_Q=qs, mat_K=ks, mat_V=vs)
rs, rs.shape

before kernel dispatch - float16!
B: 1, NH: 1, S: 16, DH: 4
blocksxy: 1-1, threads: 4-4
In Kernel: gdim.x: 1, gdim.y: 1, gdim.z: 1, bdim.x: 4, bdim.y: 4
In Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
qTIdx=0|kvTIdx=0: qTile[3][3] = 0.150024
qTIdx=0|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=0|kvTIdx=0: sTile[3][0](3) = 0.540039
qTIdx=0|kvTIdx=0: qTile[7][3] = 0.310059
qTIdx=0|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=0|kvTIdx=0: sTile[7][0](3) = 1.179688
qTIdx=1|kvTIdx=0: qTile[3][3] = 0.469971
qTIdx=1|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=1|kvTIdx=0: sTile[3][0](3) = 1.819336
qTIdx=1|kvTIdx=0: qTile[7][3] = 0.629883
qTIdx=1|kvTIdx=0: kTile[0][3] = 1.000000
qTIdx=1|kvTIdx=0: sTile[7][0](3) = 2.460938


(tensor([[[[ 0.9604,  0.9604,  0.9604,  0.9604],
           [ 3.5176,  3.5176,  3.5176,  3.5176],
           [ 6.0781,  6.0781,  6.0781,  6.0781],
           [ 8.6328,  8.6328,  8.6328,  8.6328],
           [11.1953, 11.1953, 11.1953, 11.1953],
           [13.7578, 13.7578, 13.7578, 13.7578],
           [16.3125, 16.3125, 16.3125, 16.3125],
           [18.8750, 18.8750, 18.8750, 18.8750],
           [22.4062, 22.4062, 22.4062, 22.4062],
           [27.5156, 27.5156, 27.5156, 27.5156],
           [32.6250, 32.6250, 32.6250, 32.6250],
           [37.7500, 37.7500, 37.7500, 37.7500],
           [42.8750, 42.8750, 42.8750, 42.8750],
           [48.0000, 48.0000, 48.0000, 48.0000],
           [53.0625, 53.0625, 53.0625, 53.0625],
           [58.2500, 58.2500, 58.2500, 58.2500]]]], device='cuda:0', dtype=torch.float16),
 torch.Size([1, 1, 16, 4]))

### Matmul kernel Test from vlstm_v2

In [10]:
# from src.vlstm_v3.interface import testkernel, copykernel, mmkernelv1

In [11]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [12]:
S = 8 # sequence length
DH = 8 # hidden size

In [13]:
matA = torch.arange((2*S * DH), device=DEVICE, dtype=DTYPE).reshape((2*S, DH))
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape, matA, matA.sum(-1)

(torch.Size([16, 8]),
 torch.Size([8, 8]),
 tensor([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.],
         [  8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.],
         [ 16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.],
         [ 24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.],
         [ 32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.],
         [ 40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.],
         [ 48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.],
         [ 56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.],
         [ 64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.],
         [ 72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.],
         [ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.],
         [ 88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.],
         [ 96.,  97.,  98.,  99., 100., 101., 102., 103.],
         [104., 105., 106., 107., 108., 109., 110., 111.],
         [112., 113., 114., 115., 116., 117., 118., 119.],
         [120., 121., 122., 123., 124., 125., 126., 127.]], device='cuda

In [14]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

(tensor([[ 28.,  28.,  28.,  28.,  28.,  28.,  28.,  28.],
         [ 92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.],
         [156., 156., 156., 156., 156., 156., 156., 156.],
         [220., 220., 220., 220., 220., 220., 220., 220.],
         [284., 284., 284., 284., 284., 284., 284., 284.],
         [348., 348., 348., 348., 348., 348., 348., 348.],
         [412., 412., 412., 412., 412., 412., 412., 412.],
         [476., 476., 476., 476., 476., 476., 476., 476.],
         [540., 540., 540., 540., 540., 540., 540., 540.],
         [604., 604., 604., 604., 604., 604., 604., 604.],
         [668., 668., 668., 668., 668., 668., 668., 668.],
         [732., 732., 732., 732., 732., 732., 732., 732.],
         [796., 796., 796., 796., 796., 796., 796., 796.],
         [860., 860., 860., 860., 860., 860., 860., 860.],
         [924., 924., 924., 924., 924., 924., 924., 924.],
         [988., 988., 988., 988., 988., 988., 988., 988.]], device='cuda:0', dtype=torch.bfloat16),
 torch.Size([16

In [15]:
matA.is_contiguous(), matB.is_contiguous()

(True, True)

In [16]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

NameError: name 'mmkernelv1' is not defined

In [ ]:
matA[9]

In [ ]:
matA[9].cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cuda:0')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float32, device=torch.device('cpu')).cumsum(-1)

In [ ]:
torch.arange(72,80, dtype=torch.float16, device=torch.device('cuda:0')).cumsum(-1)

In [ ]:
# cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
# cu_out, cu_out.shape

In [ ]:
# mat @ mat.T @ mat

### pytorch version

In [ ]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [ ]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))